1. A partir del archivo csv Case, determine las tres ciudades con más casos confirmados de la enfermedad. La salida debe contener tres columnas: provincia, ciudad y casos confirmados. El resultado debe contener exactamente los tres nombre de ciudades con más casos confirmados ya que no se admiten otros valores.

2. Cree un dataframe a partir del archivo csv PatientInfo. Asegúrese de que su dataframe no contenga pacientes duplicados.

    a. ¿Cuántos pacientes tienen informado por quién se contagiaron(columna infected_by)? Obtenga solo los pacientes que tengan informado por quién se contagiaron.

    b. A partir de la salida del inciso anterior obtenga solo los pacientes femeninos. La salida no debe contener las columnas released_date y deceased_date.

    c. Establezca el número de particiones del dataframe resultante del inciso anterior en dos. Escriba el dataframe resultante en un archivo parquet. La salida debe estar particionada por la provincia y el modo de escritura debe ser overwrite.

In [ ]:
import findspark

findspark.init()

from pyspark.sql import SparkSession, Row
from pyspark import SparkContext
from pyspark.sql.types import StructType, StructField, IntegerType, StringType, FloatType, DateType
from pyspark.sql.functions import col, countDistinct, desc, asc
from pyspark.storagelevel import StorageLevel

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

In [ ]:
# 1
case_df = spark.read.csv(
    r"../../data/s7_data/Case.csv",
    header=True,
    inferSchema=True,
).withColumnRenamed(
    " case_id",
    "case_id"
)

clean_case_df = (
    case_df.dropna(
        subset=["city"],
    )
    .filter(
        (col("city") != "-") &
        (col("city") != "from other city")
    )
    .select(
        col("province"),
        col("city"),
        col("confirmed"),
    )
    .orderBy(
        col("confirmed").desc()
    )
    .limit(3)
)
clean_case_df.show()

In [ ]:
# 2
patient_df = spark.read.csv(
    r"../../data/s7_data/PatientInfo.csv",
    header=True,
    inferSchema=True,
).dropDuplicates()

In [51]:
# a
infected_by = (
    patient_df
    .dropna(
        subset=["infected_by"],
    )
)

# infected_by.show(
#     n=infected_by.count(),
#     truncate=False,
# )

infected_by.count()

1346

In [ ]:
# b
female_infected = (
    infected_by
    .filter(
        (col("sex") == "female"),
    )
    .drop(
        "released_date",
        "deceased_date"
    )
)

female_infected.show()

In [ ]:
# c
female_infected.coalesce(2)

female_infected.write.partitionBy("province").parquet(
    path=r"./data/s7_out",
    mode="overwrite"
)